# Description

This exercise has a similar setup to the first exercise.  It also works with 1000 files on the local disk, each of them named like `tmp-?????.numbers`, where each contains 20 whole numbers on separate lines.  As with the prior exercise, the operations you will perform are fast enough that a purely sequential approach is faster than a threaded one, but the balance would change with larger files and/or more complex operations.

In four threads you should perform the following operations. Here the index positions are their 1-based position.  E.g., if this is the head of one of the files:

```
5
32
4
18
```

Index 2 is 32; index 3 is 4; etc.

* Read index 2 and 10 from each file, then multiply them together;
* Read index 5 and 10 from each file, then substract index 5 from index 10;
* Read index 11 and 15 from each file, then take the floor division of 11 by 15;
* Read index 5 and 19 from each file, as x and y, then calculate $x^2 + y^2$.

The function `sum_of_ops()` should return the total of all the quantities caclulated in the threads, along with a list of the partial results from each thread, in order.

During testing or experimentation, you are likely to start the threads, but be sure to leave each of them in their created-but-not-started state before you run the tests.

# Setup



In [1]:
from threading import Thread, current_thread
from generate import create_files
from time import sleep

create_files('lesson-2')

t1 = Thread(target=sleep, args=(1,), name="mult2,10")
t2 = Thread(target=sleep, args=(1,), name="diff5,10")
t3 = Thread(target=sleep, args=(1,), name="fdiv11,15")
t4 = Thread(target=sleep, args=(1,), name="hypot5,19")

def sum_of_ops(t1: Thread, t2: Thread, t3: Thread, t4: Thread) -> int:
    [t.start() for t in (t1, t2, t3, t4)]
    t1_result = [12, 34, 56]
    t2_result = [23, 45, 67]
    t3_result = [34, 56, 78]
    t4_result = [45, 67, 89]
    return 606, t1_result, t2_result, t3_result, t4_result

# Solution

In [2]:
from functools import partial
from itertools import chain
from glob import glob
calculations = {}

def getcalc(i, j, fn):
    global calculations
    name = current_thread().name
    calculations[name] = []
    for fname in glob('tmp-*.numbers'):
        nums = [int(n) for n in open(fname)]
        calculations[name].append(fn(nums[i-1], nums[j-1]))

t1 = Thread(target=partial(getcalc, 2, 10, lambda n, m: n+m))
t2 = Thread(target=partial(getcalc, 5, 10, lambda n, m: m-n))
t3 = Thread(target=partial(getcalc, 11, 15, lambda n, m: n//m))
t4 = Thread(target=partial(getcalc, 5, 19, lambda n, m: n**2 + m**2))

def sum_of_ops(t1, t2, t3, t4):
    [t.start() for t in (t1, t2, t3, t4)]
    [t.join() for t in (t1, t2, t3, t4)]
    total = sum(chain.from_iterable(calculations.values()))
    return total, *calculations.values()

# Test Cases

In [3]:
def test_types():
    assert callable(sum_of_ops)
    for t in (t1, t2, t3, t4):
        assert isinstance(t, Thread)
    
test_types()

In [4]:
def test_thread_state():
    for t in (t1, t2, t3, t4):
        assert not t.is_alive() and not t.ident, t
        
test_thread_state()

In [5]:
def test_results():
    from math import prod
    total, l1, l2, l3, l4 = sum_of_ops(t1, t2, t3, t4)
    assert total == 6748263, f"Wrong total {total}"
    for lst in (l1, l2, l3, l4):
        assert len(lst) == 1000, "Partial results of wrong length"
    assert prod(l1) % 78233 == 8953, "Wrong list 1 calculations"
    assert sum(l2) == -2089, "Wrong list 2 calculations"
    assert sum(l3) == 2359, "Wrong list 3 calculations"
    assert prod(l4) % 82171 == 75175, "Wrong list 4 calculations"
    
test_results()